In [ ]:
import cv2
import glob
import pickle
import numpy as np
import matplotlib.image as mpimg
import matplotlib.pyplot as plt
%matplotlib inline

In [ ]:
def grayscale(img):
    return cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)

In [ ]:
def camera_calibration():
    # prepare object points, like (0,0,0), (1,0,0), (2,0,0) ....,(6,5,0)
    objp = np.zeros((6*9,3), np.float32)
    objp[:,:2] = np.mgrid[0:9,0:6].T.reshape(-1,2)

    # Arrays to store object points and image points from all the images.
    objpoints = [] # 3d points in real world space
    imgpoints = [] # 2d points in image plane.

    # Make a list of calibration images
    images = glob.glob('camera_cal/calibration*.jpg')

    # Step through the list and search for chessboard corners
    for fname in images:
        img = cv2.imread(fname)
        gray = grayscale(img)

        # Find the chessboard corners
        ret, corners = cv2.findChessboardCorners(
            image = gray,
            patternSize = (9,6),
            corners = None
        )

        # If found, add object points, image points
        if ret == True:
            objpoints.append(objp)
            imgpoints.append(corners)

            # Draw and display the corners
            img = cv2.drawChessboardCorners(
                image = img,
                patternSize = (9,6),
                corners = corners,
                patternWasFound = ret)
    
    ret, mtx, dist, rvec, tvecs = cv2.calibrateCamera(
        objectPoints = objpoints,
        imagePonts = imgpoints,
        imageSize = gray.shape[::-1],
        cameraMatrix = None,
        distCoeffs = None
    )

    cam_calib = {"cam_matrix": mtx, "dist_coeffs": dist}
    with open("output_images/cam_calib.p", "wb") as f:
        pickle.dump(
            obj = cam_calib,
            file = f
        )
            

In [ ]:
def undistort_image(img):
    dist_pickle = pickle.load( open("output_images/cam_calib.p", "rb") )
    mtx = dist_pickle["cam_matrix"]
    dist = dist_pickle["dist_coeffs"]
    dst = cv2.undistort(
        src = img,
        cameraMatrix = mtx,
        distCoeffs = dist,
        dst = None,
        newCameraMatrix = mtx
    
    )
    return dst

In [ ]:
def warper(img):

    src = np.float32([[200, 720], [1100, 720], [595, 450], [685, 450]])
    dst = np.float32([[300, 720], [980, 720], [300, 0], [980, 0]])

    M = cv2.getPerspectiveTransform(src, dst)
    M_inv = cv2.getPerspectiveTransform(dst, src)
    warped = cv2.warpPerspective(
        src = img,
        M = M,
        dsize = (img.shape[1], img.shape[0]),
        flags = cv2.INTER_LINEAR
    )  # keep same size as input image
    
    unwarped = cv2.warpPerspective(
        src = warped,
        M = M_inv,
        dsize = (warped.shape[1], warped.shape[0]),
        flags = cv2.INTER_LINEAR
    )  # keep same size as input image

    return warped, unwarped


In [ ]:
def apply_sobel(gray_img, direction):
    return cv2.Sobel(
        source = gray_img,
        ddepth = cv2.CV_64F,
        dx = int('x' in direction),
        dy = int('y' in direction)
    )


In [ ]:
def abs_sobel(gray_img, direction):
    return (apply_sobel(gray_img, 'x') ** 2 + apply_sobel(gray_img, 'y') ** 2) ** 0.5 if direction == 'xy' else np.absolute(apply_sobel(gray_img, direction)) 

In [ ]:
def scaled_sobel(gray_img, direction):
    abs_sob = abs_sobel(gray_img, direction)
    return np.uint8(255 * abs_sob / np.max(abs_sob))

In [ ]:
def binary(range_param, threshold):
    binary = np.zeros_like(range_param)
    binary[(threshold[0] <= range_param) & (range_param <= threshold[1])] = 1
    return binary

In [ ]:
def process_image(img):
    threshold = (50, 255)
    gray = grayscale(img)

    xbin = binary(
        range_param = scaled_sobel(gray, 'x'),
        threshold = threshold
    )

    magnitude_bin = binary(
        range_param = scaled_sobel(gray, 'xy'),
        threshold = threshold
    )

    direction_bin = binary(
        range_param = np.arctan2(abs_sobel(gray, 'y'), abs_sobel(gray, 'x')),
        threshold = (.7, 1.3)
    )

    hls = cv2.cvtColor(img, cv2.COLOR_BGR2HLS)
    
    sbin = binary(
        range_param = hls[:,:,2],
        threshold = (180, 255)
    )

    combined = np.zeros_like(direction_bin)
    combined[((xbin == 1) | ((magnitude_bin == 1) & (direction_bin == 1))) | (sbin == 1)] = 1

    return combined


In [ ]:
def detect_lanes(warped_binary_img):
    histogram = np.sum(warped_binary_img[warped_binary_img.shape[0] // 2:,:], axis = 0)
    out_img = np.dstack((warped_binary_img, warped_binary_img, warped_binary_img)) * 255
    midpoint = np.int(histogram.shape[0] // 2)
    leftx_base = np.argmax(histogram[:midpoint])
    rightx_base = np.argmax(histogram[midpoint:]) + midpoint

    nwindows = 9
    margin = 100
    minpix = 50

    windows_height = np.int(warped_binary_img.shape[0] // nwindows)

    nonzero = warped_binary_img.nonzero()
    nonzero_x = np.array(nonzero[1])
    nonzero_y = np.array(nonzero[0])

    leftx_current = leftx_base
    rightx_current = rightx_vase

    left_lane_indices = []
    right_lane_indices = []

    for window in range(nwindows):
        win_y_low = warped_binary_img.shape[0] - (window + 1) * windows_height
        win_y_high = warped_binary_img.shape[0] - window * windows_height

        winx_left_low = leftx_current - margin
        winx_left_high = leftx_current + margin
        winx_right_low = rightx_current - margin
        winx_right_high = rightx_current + margin

        #Draw the windows on the vizualisation image
        cv2.rectangle(
            img = out_img,
            pt1 = (win_xleft_low, win_y_low),
            pt2 = (win_xleft_high, win_y_high),
            color = (0, 255, 0),
            thickness = 2
        )

        cv2.rectangle(
            img = out_img,
            pt1 = (win_xright_low, win_y_low),
            pt2 = (win_xright_high, win_y_high),
            color = (0, 255, 0),
            thickness = 2
        )


In [ ]:
camera_calibration()
images = glob.glob("test_images/*.jpg")

for fname in images:
    file_name = fname[fname.rfind('/', 0) + 1:]
    img = cv2.imread(fname)
    dist = undistort_image(img) # Looks like there's a problem here
    bin_img = process_image(dist)
    warp_img, unwarped_img = warper(bin_img)

    #plt.figure()
    #plt.imshow(warper(img, src, dst))
    f, (ax1, ax2, ax3) = plt.subplots(
        nrows = 1,
        ncols = 3,
        figsize = (24, 9)
    )
    f.tight_layout()
    ax1.imshow(img)
    ax1.set_title("Original", fontsize = 50)
    ax2.imshow(unwarped_img)
    ax2.set_title("Unwarper image", fontsize = 50)
    ax3.imshow(warp_img)
    ax3.set_title("Warper image", fontsize = 50)
    plt.subplots_adjust(left = 0., right = 1., top = .9, bottom = 0.)
    #cv2.imwrite("output_images/" + file_name, bin_img)

    #break